# Mojtaba Kanani

In [1]:
import pandas as pd
import numpy as np
from hazm import *

In [2]:
from __future__ import unicode_literals

In [3]:
train = pd.read_csv('datasets/train_comments.csv')

In [4]:
train.head()

,id,title,comment,rate,verification_status
0,0,کیفیت و حجم صدای عااااالی,این محصول توی بازار اصلاااا پیدا نمیشه من کل ت...,100.0,0
1,1,شش ماه مصرف!!,دوبار از این مدل گرفتم اولاش خوبه ولی بعد از ش...,5.0,0
2,3,بررسی کمی و کیفی,برای من بسیار مناسب و خریدش در شگفت انگیز حتما...,0.0,0
3,4,بسته بندی ضعیف,ظاهر بامزه ای داره ولی عکسش شبیه خودش نیست جنس...,60.0,0
4,6,نسبت به قیمت خوبه,خوبه نسبت به قیمت شگفت انگیزش,45.0,0


# Preprocessing

In [5]:
train['comment'] = train['comment'].astype(str)

## Normalization

In [6]:
normalizer = Normalizer()

In [7]:
train['Normalized'] = train['comment'].apply(lambda x:normalizer.normalize(x))

In [8]:
tagger = POSTagger(model='resources/postagger.model')

In [ ]:
train['tagtokenized'] = train['Normalized'].apply(lambda x:tagger.tag(word_tokenize(x)))

## Stemming

In [10]:
stemmer = Stemmer()

In [11]:
def stemthis(tagtokenized):
    empty = []
    listt = tagtokenized
    for i in range(len(listt)):
        empty.append((stemmer.stem(listt[i][0]),listt[i][1]))
    return empty

In [12]:
train['stemmed'] = train['tagtokenized'].apply(lambda x:stemthis(x))

## Lemmitization

In [13]:
lemmatizer = Lemmatizer()

In [14]:
def lemmatizethis(stemmed):
    empty = []
    listt = stemmed
    for i in range(len(listt)):
        if listt[i][1]=='V':
            empty.append((lemmatizer.lemmatize(listt[i][0]),'V'))
        else:
            empty.append(listt[i])
    return empty

In [15]:
train['lemmatized'] = train['stemmed'].apply(lambda x:lemmatizethis(x))

## Stop Words

In [16]:
listofstopwords = """دیگران
همچنان
مدت
چیز
سایر
جا
طی
کل
کنونی
بیرون
مثلا
کامل
کاملا
آنکه
موارد
واقعی
امور
اکنون
بطور
بخشی
تحت
چگونه
عدم
نوعی
حاضر
وضع
مقابل
کنار
خویش
نگاه
درون
زمانی
بنابراین
تو
خیلی
بزرگ
خودش
جز
اینجا
مختلف
توسط
نوع
همچنین
آنجا
قبل
جناح
اینها
طور
شاید
ایشان
جهت
طریق
مانند
پیدا
ممکن
کسانی
جای
کسی
غیر
بی
قابل
درباره
جدید
وقتی
اخیر
چرا
بیش
روی
طرف
جریان
زیر
آنچه
البته
فقط
چیزی
چون
برابر
هنوز
بخش
زمینه
بین
بدون
استفاد
همان
نشان
بسیاری
بعد
عمل
روز
اعلام
چند
آنان
بلکه
امروز
تمام
بیشتر
آیا
برخی
علیه
دیگری
ویژه
گذشته
انجام
حتی
داده
راه
سوی
ولی
زمان
حال
تنها
بسیار
یعنی
عنوان
همین
هبچ
پیش
وی
یکی
اینکه
وجود
شما
پس
چنین
میان
مورد
چه
اگر
همه
نه
دیگر
آنها
باید
هر
او
ما
من
تا
نیز
اما
یک
خود
بر
یا
هم
را
این
با
آن
برای
و
در
به
از
که
""".split('\n')

In [17]:
def removeStopWords(lemmatized):
    listt = lemmatized
    empty = []
    for i in range(len(listt)):
        if listt[i][0] not in listofstopwords:
            empty.append(listt[i])
    return empty

In [18]:
train['removedstopwords'] = train['lemmatized'].apply(lambda x:removeStopWords(x))

# Text to Vector

### Making Corpus for Vectorizing

In [19]:
def removeextra(removedstopwords):
    empty = []
    listt = removedstopwords
    for i in range(len(listt)):
        empty.append(listt[i][0])
    return empty

In [20]:
train['extraremoved'] = train['removedstopwords'].apply(lambda x:removeextra(x))

In [23]:
train.head()

,id,title,comment,rate,verification_status,Normalized,tagtokenized,stemmed,lemmatized,removedstopwords,extraremoved
0,0,کیفیت و حجم صدای عااااالی,این محصول توی بازار اصلاااا پیدا نمیشه من کل ت...,100.0,0,این محصول توی بازار اصلاااا پیدا نمیشه من کل ت...,"[(این, DET), (محصول, N), (توی, Pe), (بازار, N)...","[(این, DET), (محصول, N), (تو, Pe), (بازار, N),...","[(این, DET), (محصول, N), (تو, Pe), (بازار, N),...","[(محصول, N), (بازار, N), (اصلاااا, N), (نمیشه,...","[محصول, بازار, اصلاااا, نمیشه, تهر, رو, گ, واق..."
1,1,شش ماه مصرف!!,دوبار از این مدل گرفتم اولاش خوبه ولی بعد از ش...,5.0,0,دوبار از این مدل گرفتم اولاش خوبه ولی بعد از ش...,"[(دوبار, N), (از, P), (این, DET), (مدل, N), (گ...","[(دوبار, N), (از, P), (این, DET), (مدل, N), (گ...","[(دوبار, N), (از, P), (این, DET), (مدل, N), (گ...","[(دوبار, N), (مدل, N), (گرف, V), (اولا, N), (خ...","[دوبار, مدل, گرف, اولا, خوبه, ول, ش, ماه, باد,..."
2,3,بررسی کمی و کیفی,برای من بسیار مناسب و خریدش در شگفت انگیز حتما...,0.0,0,برای من بسیار مناسب و خریدش در شگفت انگیز حتما...,"[(برای, Pe), (من, PRO), (بسیار, ADV), (مناسب, ...","[(برا, Pe), (من, PRO), (بسیار, ADV), (مناسب, A...","[(برا, Pe), (من, PRO), (بسیار, ADV), (مناسب, A...","[(برا, Pe), (مناسب, AJ), (خرید, N), (شگف, AJ),...","[برا, مناسب, خرید, شگف, انگیز, حتما, پیشنهاد, ..."
3,4,بسته بندی ضعیف,ظاهر بامزه ای داره ولی عکسش شبیه خودش نیست جنس...,60.0,0,ظاهر بامزه‌ای داره ولی عکسش شبیه خودش نیست جنس...,"[(ظاهر, Ne), (بامزه‌ای, AJ), (داره, V), (ولی, ...","[(ظاهر, Ne), (بامزه, AJ), (داره, V), (ول, CONJ...","[(ظاهر, Ne), (بامزه, AJ), (داره, V), (ول, CONJ...","[(ظاهر, Ne), (بامزه, AJ), (داره, V), (ول, CONJ...","[ظاهر, بامزه, داره, ول, عکس, شبیه, نیس, جنس, چ..."
4,6,نسبت به قیمت خوبه,خوبه نسبت به قیمت شگفت انگیزش,45.0,0,خوبه نسبت به قیمت شگفت انگیزش,"[(خوبه, V), (نسبت, N), (به, P), (قیمت, Ne), (ش...","[(خوبه, V), (نسب, N), (به, P), (قیم, Ne), (شگف...","[(خوبه, V), (نسب, N), (به, P), (قیم, Ne), (شگف...","[(خوبه, V), (نسب, N), (قیم, Ne), (شگف, AJe), (...","[خوبه, نسب, قیم, شگف, انگیز]"


In [21]:
def makecorpus(notcorpus):
    listt = notcorpus
    string = ''
    for i in range(len(listt)):
        string += listt[i]
        if i!=len(listt):
            string += ' '
    return string

### x_train , y_train , x_test

In [44]:
x_train = pd.DataFrame(train['extraremoved'].apply(lambda x:makecorpus(x)))
x_train.index.name = 'id'
x_train.columns = ['comment']

In [46]:
y_train = pd.read_csv('datasets/train_comments.csv',index_col = 0 ,usecols=['id','verification_status'])
x_test = pd.read_csv('datasets/test_nolabel_comments.csv',index_col=0 ,usecols=['id','comment'])

In [47]:
y_train = y_train.astype('int')
x_train = x_train.fillna('0')
x_test = x_test.fillna('0')

In [48]:
x_train.head()

,comment
id,
0,محصول بازار اصلاااا نمیشه تهر رو گ واقعا محصول...
1,دوبار مدل گرف اولا خوبه ول ش ماه باد میکنه صدد...
2,برا مناسب خرید شگف انگیز حتما پیشنهاد می‌کن
3,ظاهر بامزه داره ول عکس شبیه نیس جنس چوب معمول ...
4,خوبه نسب قیم شگف انگیز


In [49]:
y_train.head()

,verification_status
id,
0,0
1,0
3,0
4,0
6,0


In [50]:
x_test.head()

,comment
id,
2267,خیلی شیک و کاربردی.\r\nاز بسته بندی شیک و ساده...
69180,فوق العادست حتما پیشنهاد میکنم.مطمئن باشید پشی...
130621,توی تن خوب به نظر میاد
164512,من فکر میکردم که 2تا بلندگو داشته باشه و وقتی ...
105086,همه چیز این گوشی خوبه فقط دور حاشیه صفحه نمایش...


### Evaluation Functions

In [51]:
def accuracy(list1,list2):
    A = list(list1)
    B = list(list2)
    TPTN = 0
    if len(A)==len(B):
        for i in range (len(A)):
            if A[i]==B[i]:
                TPTN +=1
    accuracy = TPTN / len(A)
    return accuracy

In [52]:
def precision(list1,list2):
    A = list(list1)
    B = list(list2)
    TP = 0
    FP = 0
    if len(A)==len(B):
        for i in range (len(A)):
            if A[i]==B[i] and A[i]==1:
                TP +=1
            if A[i]==0 and B[i]==1:
                FP +=1
    TPFP = TP + FP
    precision = TP/TPFP
    return precision

In [53]:
def recall(list1,list2):
    A = list(list1)
    B = list(list2)
    TP = 0
    FN = 0
    if len(A)==len(B):
        for i in range (len(A)):
            if A[i]==B[i] and A[i]==1:
                TP +=1
            if A[i]==1 and B[i]==0:
                FN +=1
    TPFN = TP + FN
    recall = TP/TPFN
    return recall

In [86]:
def f1score(list1,list2):
    precisiontemp = precision(list1,list2)
    recalltemp = recall(list1,list2)
    f1score = 2*(precisiontemp*recalltemp)/(precisiontemp+recalltemp)
    return f1score

## Two Vectorization Methods

### 1- Count Vectorizer

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

In [58]:
cv1 = CountVectorizer()

In [59]:
x_traincv1 = cv1.fit_transform(x_train['comment'])

In [60]:
x_traincv1.get_shape()

(162000, 69519)

In [61]:
x_traincv1.sum()

3406637

### 2- TFIDF Vectorizer

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [64]:
cv2 = TfidfVectorizer(min_df=1)

In [65]:
x_traincv2 = cv2.fit_transform(x_train['comment'])

In [66]:
x_traincv2.get_shape()

(162000, 69519)

In [67]:
x_traincv2.sum()

563788.1873895341

# Classification

## A - MultinomialNB

In [70]:
from sklearn.naive_bayes import MultinomialNB

### 1- Count Vectorizer

#### Test

In [73]:
mnb1 = MultinomialNB()

In [75]:
mnb1.fit(x_traincv1, y_train['verification_status'].values.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [76]:
x_testcv1 = cv1.transform(x_test['comment'])

In [77]:
mnbtest1 = mnb1.predict(x_testcv1)

In [78]:
mnbtest1.sum()

4856

#### Train

In [80]:
mnbtrain1 = mnb1.predict(x_traincv1)

In [81]:
mnbtrain1.sum()

21580

#### Evaluation

In [82]:
accuracy(y_train['verification_status'],mnbtrain1)

0.8935679012345679

In [83]:
precision(y_train['verification_status'],mnbtrain1)

0.7252085264133457

In [84]:
recall(y_train['verification_status'],mnbtrain1)

0.5804465544099102

In [87]:
f1score(y_train['verification_status'],mnbtrain1)

0.6448024391248816

### 2- TFIDF Vectorizer

#### Test

In [90]:
mnb2 = MultinomialNB()

In [91]:
mnb2.fit(x_traincv2, y_train['verification_status'].values.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [92]:
x_testcv2 = cv2.transform(x_test['comment'])

In [93]:
mnbtest2 = mnb2.predict(x_testcv2)

In [94]:
mnbtest2.sum()

558

#### Train

In [96]:
mnbtrain2 = mnb2.predict(x_traincv2)

In [97]:
mnbtrain2.sum()

5413

#### Evaluation

In [98]:
accuracy(y_train['verification_status'],mnbtrain2)

0.8646234567901234

In [99]:
precision(y_train['verification_status'],mnbtrain2)

0.9647145760206909

In [100]:
recall(y_train['verification_status'],mnbtrain2)

0.19367999406572212

In [116]:
f1score(y_train['verification_status'],mnbtrain2)

0.32259459459459455

## B - Logistic Regression

In [102]:
from sklearn.linear_model import LogisticRegression

### 1- Count Vectorizer

#### Test

In [106]:
LR1 = LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter=1000).fit(x_traincv1,y_train['verification_status'])

In [107]:
lrtest1 = LR1.predict(x_testcv1)

In [108]:
lrtest1.sum()

2715

#### Train

In [110]:
lrtrain1 = LR1.predict(x_traincv1)

In [111]:
lrtrain1.sum()

17583

#### Evaluation

In [112]:
accuracy(y_train['verification_status'],lrtrain1)

0.919746913580247

In [113]:
precision(y_train['verification_status'],lrtrain1)

0.8970027867826879

In [114]:
recall(y_train['verification_status'],lrtrain1)

0.5849714412877383

In [115]:
f1score(y_train['verification_status'],lrtrain1)

0.7081378381412055

### 2- TFIDF Vectorizer

#### Test

In [119]:
LR2 = LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter=1000).fit(x_traincv2,y_train['verification_status'])

In [120]:
lrtest2 = LR2.predict(x_testcv2)

In [121]:
lrtest2.sum()

1280

#### Train

In [123]:
lrtrain2 = LR2.predict(x_traincv2)

In [124]:
lrtrain2.sum()

16652

#### Evaluation

In [125]:
accuracy(y_train['verification_status'],lrtrain2)

0.907283950617284

In [126]:
precision(y_train['verification_status'],lrtrain2)

0.8585755464809032

In [127]:
recall(y_train['verification_status'],lrtrain2)

0.530264817150063

In [128]:
f1score(y_train['verification_status'],lrtrain2)

0.6556151694410051

## C - SVM

In [130]:
from sklearn.svm import SVC

### 1- Count vectorizer

#### Test

In [ ]:
svc1 = SVC(kernel = 'linear',class_weight='balanced',random_state=0).fit(x_traincv1,y_train['verification_status'])

In [133]:
# time kheili ziadi tool keshid va monsrf shodim vali mse qbli ha piade sazi mishe.